# **Setup**
## Recompile Cython Extensions

In [ ]:
%cd /scratch/edk202/ngram-prep

%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8

%pip install -e . --no-build-isolation -q

## Imports

In [1]:
# Auto-reload edited scripts
%load_ext autoreload
%autoreload 2

# NLTK resources
from nltk.corpus import stopwords; stopwords = set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer; lemmatizer = WordNetLemmatizer()

# Ngram acquisition functions
from ngram_prep.ngram_acquire import download_and_ingest_to_rocksdb
from ngram_prep.ngram_acquire.logger import setup_logger

# Ngram processing functions
from pathlib import Path
from ngram_prep.ngram_filter.config import PipelineConfig, FilterConfig
from ngram_prep.ngram_filter.pipeline.orchestrator import build_processed_db
from ngram_prep.parallel import SplitMonitorConfig
from ngram_prep.utilities.peek import db_peek, db_peek_prefix

## Set up logging to file

In [2]:
setup_logger(
    db_path="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

PosixPath('/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/ngram_download_20251012_104941.log')

# **Download 5-Grams and Ingest to RocksDB**

In [2]:
download_and_ingest_to_rocksdb(
    ngram_size=1,
    repo_release_id="20200217",
    repo_corpus_id="eng",
    db_path_stub="/vast/edk202/NLP_corpora/Google_Books/",
    file_range=(0, 19422),
    random_seed=98,
    workers=39,
    use_threads=False,
    ngram_type="tagged",
    overwrite_db=True,
    write_batch_size=100_000,
    open_type="write:packed24",
    compact_after_ingest=True
)

ValueError: Invalid file range (0, 19422). Available: 0..23

# **Run Processing Pipeline**

In [4]:
src_db = Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db")
dst_db = src_db.parent / "5grams_processed.db"
tmp_dir = src_db.parent / "processing_tmp"
whitelist_dir = src_db.parent.parent / "1gram_files" / "1grams_processed.db" / "whitelist.txt"

filter_config = FilterConfig(
    stop_set=stopwords,
    lemma_gen=lemmatizer,
    whitelist_path=whitelist_dir
)

split_monitor_config = SplitMonitorConfig(
    check_interval=2,
    starvation_threshold=4
)

pipeline_config = PipelineConfig(
    src_db=src_db,
    dst_db=dst_db,
    tmp_dir=tmp_dir,
    num_workers=30,
    num_ingest_workers=10,
    num_initial_work_units=30,
    mode="resume",
    enable_ingest=True,
    delete_after_ingest=True,
    compact_after_ingest=False,
    progress_every_s=60.0,
    max_items_per_bucket=100_000,
    max_bytes_per_bucket=128 * 1024 * 1024,
    split_monitor_config=split_monitor_config
)

build_processed_db(pipeline_config, filter_config)

N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Pipeline
Run mode:               resume
Ingest after filtering: True

Workers
Num Workers:        30
Initial work units: 30
Dynamic splitting:  Enabled
Profiles:           read=read:packed24, write=write:packed24
Buffer:             100,000 items, 128.00 MB

Files
Source: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
Destination: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Input whitelist: ..._corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db/whitelist.txt
  All tokens (min count: 1)
Output whitelist: None
Loading whitelist...
Loaded 40,000 tokens

Phase 1: Creating work units...
═══════════════════════════════════════════════════════════════════════════════════════════

# **Inspect the Procesed Database**
## `db_head`: Print the first _N_ key–value pairs

In [5]:
LGdb_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_head(db_path, key_format="utf-8", value_format="packed", n=5)

NameError: name 'db_head' is not defined

## `db_peek`: Print _N_ key-value pairs starting at the specified key

In [ ]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_peek(db_path, start_key=b"quick brown <UNK> <UNK> <UNK>", key_format="utf-8", value_format="packed", n=5)


## `db_peek_prefix`: Print key-value pairs containing the specified prefix

In [ ]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_peek_prefix(db_path, prefix=b"<UNK> united state <UNK> <UNK>", key_format="utf-8", value_format="summary", n=1)